In [ ]:
# print("="*60)
# print("APPLE STOCK PRICE PREDICTION SYSTEM")
# print("="*60)

APPLE STOCK PRICE PREDICTION SYSTEM


In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.svm import SVR
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# import matplotlib.pyplot as plt
# import seaborn as sns
# from datetime import datetime
# print("✓ Libraries imported successfully")



✓ Libraries imported successfully


In [50]:
print("\nStep 2: Loading Apple stock price data...")
try:
    # Load the data
    df = pd.read_csv('AppleStock.csv')
    print(f"✓ Data loaded successfully")
    print(f"  - File: AppleStock.csv")
    print(f"  - Shape: {df.shape[0]} rows, {df.shape[1]} columns")
    print(f"  - Columns: {list(df.columns)}")
    
    # Display first few rows
    print("\n  First 5 rows of data:")
    print(df.head())
    
except FileNotFoundError:
    print("✗ ERROR: 'AppleStock.csv' not found!")
    print("Please make sure the file is in the same directory.")
    exit()


Step 2: Loading Apple stock price data...
✓ Data loaded successfully
  - File: AppleStock.csv
  - Shape: 2727 rows, 7 columns
  - Columns: ['Date', 'Open', 'High', 'Low', 'Close\xa0', 'Adj Close\xa0', 'Volume']

  First 5 rows of data:
        Date   Open   High    Low  Close   Adj Close        Volume
0  23-Jan-15  28.08  28.44  27.88   28.25       25.05  185,859,200
1  26-Jan-15  28.43  28.59  28.20   28.27       25.07  222,460,000
2  27-Jan-15   28.1  28.12  27.26   27.28       24.20  382,274,800
3  28-Jan-15  29.41  29.53  28.83   28.83       25.56  585,908,400
4  29-Jan-15  29.08  29.80  28.89   29.73       26.36  337,745,600


In [55]:
print("\nStep 3: Data Cleaning and Preprocessing...")

# Clean Volume column (remove commas)
if 'Volume' in df.columns:
    df['Volume'] = df['Volume'].astype(str).str.replace(',', '').astype(float)
    print("✓ Volume column cleaned")

# Convert Date column
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'])
    # Extract date features
    df['Day'] = df['Date'].dt.day
    df['Month'] = df['Date'].dt.month
    df['Year'] = df['Date'].dt.year
    print("✓ Date column converted and features extracted")



Step 3: Data Cleaning and Preprocessing...
✓ Volume column cleaned
✓ Date column converted and features extracted


In [59]:
initial_rows = len(df)
df = df.dropna()
df = df.drop_duplicates()
print(f"✓ Missing values and duplicates removed")
print(f"  - Removed {initial_rows - len(df)} rows")

print(f"\n  Data shape after cleaning: {df.shape}")
print("✓ Data preprocessing completed")

✓ Missing values and duplicates removed
  - Removed 1 rows

  Data shape after cleaning: (2726, 12)
✓ Data preprocessing completed


In [62]:
print("\nStep 4: Splitting data into training and testing sets...")

# Define features and target
features = []
for col in ['Open', 'High', 'Low', 'Volume', 'Day', 'Month', 'Year']:
    if col in df.columns:
        features.append(col)

X = df[features]
y = df['Close']  # Predicting closing price

print(f"  Features used: {features}")
print(f"  Target: Closing price")



Step 4: Splitting data into training and testing sets...


KeyError: 'Close'

In [ ]:
# Define Features (Inputs) and Target (Output)
X = df[['Date_Ordinal', 'Open', 'High', 'Low', 'Volume']]
y = df['Close']

# Split: 80% for training, 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

KeyError: 'Close'